In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, date

In [7]:
path='./data/Insatser till brand i skog och mark 2000-2020.xlsx'

In [25]:
df = pd.read_excel(path)

In [34]:
df['TotArea'] = df['arealProduktivSkogsmark_m2']+df['arealAnnanTradbevuxenMark_m2']+df['arealMarkUtanTrad_m2']
df

,ar,datum,tid,kommun,kommunKortNamn,verksamhetText,sweref99Norr,sweref99Ost,BEJBbrandorsakText,arealProduktivSkogsmark_m2,arealAnnanTradbevuxenMark_m2,arealMarkUtanTrad_m2
0,2000,2000-01-01,00:07:00,1272,Bromölla,Verksamhet inte knuten till en byggnad,6856.0,-905921.0,Fyrverkeri eller pyroteknik,0,6,0
1,2000,2000-01-01,01:04:00,1452,Tranemo,Verksamhet inte knuten till en byggnad,NaN,NaN,Okänd,0,0,5
2,2000,2000-01-02,01:05:00,2283,Sollefteå,"Lantbruk, inte bostad",NaN,NaN,Annan,0,0,15000
3,2000,2000-01-02,19:35:00,180,Stockholm,Verksamhet inte knuten till en byggnad,NaN,NaN,Okänd,0,5,0
4,2000,2000-01-03,12:37:00,180,Stockholm,Verksamhet inte knuten till en byggnad,NaN,NaN,Barns lek med eld,0,50,0
...,...,...,...,...,...,...,...,...,...,...,...,...
100302,2020,2020-12-29,01:52:00,180,Stockholm,Verksamhet inte knuten till en byggnad,6586007.0,664319.0,Avsiktlig brand,0,1,0
100303,2020,2020-12-29,18:56:00,117,Österåker,Verksamhet inte knuten till en byggnad,6599203.0,687136.0,Avsiktlig brand,0,0,1
100304,2020,2020-12-30,12:32:00,380,Uppsala,Verksamhet inte knuten till en byggnad,6637240.0,653312.0,Okänd,0,0,20
100305,2020,2020-12-31,15:17:00,1384,Kungsbacka,Verksamhet inte knuten till en byggnad,6360762.0,327099.0,Annan,0,0,1


In [35]:
for column in df:
    if df[column].isnull().any():
       print('{0} has {1} null values'.format(column, df[column].isnull().sum()))

sweref99Norr has 20718 null values
sweref99Ost has 20717 null values
